In [2]:
# Import the data sets  using pandas 
import pandas as pd 
df=pd.read_csv("C:\\Users\\balav\\Downloads\\blogs.csv")


In [3]:
# It will show the first five columns 
df.head()

,Data,Labels
0,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
1,Newsgroups: alt.atheism\nPath: cantaloupe.srv....,alt.atheism
2,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,alt.atheism
3,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
4,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...,alt.atheism


In [4]:
#It will describe the columns 
df.describe()

,Data,Labels
count,2000,2000
unique,2000,20
top,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
freq,1,100


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Data    2000 non-null   object
 1   Labels  2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


In [10]:
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

# Ensure necessary NLTK data is downloaded
nltk.download('stopwords')
nltk.download('punkt')


# Check for missing values
print("\nMissing values in the blogs data:")
display(df.isnull().sum())


# Preprocess the text data

def clean_text(text):
    # Remove punctuation and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    return text

df['Cleaned_Data'] = df['Data'].apply(clean_text)

# Tokenize the text
df['Tokens'] = df['Cleaned_Data'].apply(lambda x: x.split())

# Remove stopwords
stop_words = set(stopwords.words('english'))
df['Tokens_NoStopwords'] = df['Tokens'].apply(lambda x: [word for word in x if word not in stop_words])

# Rejoin tokens into a string for TF-IDF
df['Processed_Text'] = df['Tokens_NoStopwords'].apply(lambda x: ' '.join(x))

print("\nFirst 5 rows of data after preprocessing:")
display(df.head())

# Feature Extraction using TF-IDF
# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000) # You can adjust max_features

# Fit and transform the processed text data
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Processed_Text'])

print("\nTF-IDF matrix shape:", tfidf_matrix.shape)
print("Feature extraction complete.")

# The tfidf_matrix is now ready to be used as features for a Naive Bayes model.
# The corresponding labels are in blogs_df['Labels']


Missing values in the blogs data:


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\balav\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\balav\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Data      0
Labels    0
dtype: int64


First 5 rows of data after preprocessing:


,Data,Labels,Cleaned_Data,Tokens,Tokens_NoStopwords,Processed_Text
0,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism,path cantaloupesrvcscmuedumagnesiumclubcccmued...,"[path, cantaloupesrvcscmuedumagnesiumclubcccmu...","[path, cantaloupesrvcscmuedumagnesiumclubcccmu...",path cantaloupesrvcscmuedumagnesiumclubcccmued...
1,Newsgroups: alt.atheism\nPath: cantaloupe.srv....,alt.atheism,newsgroups altatheism\npath cantaloupesrvcscmu...,"[newsgroups, altatheism, path, cantaloupesrvcs...","[newsgroups, altatheism, path, cantaloupesrvcs...",newsgroups altatheism path cantaloupesrvcscmue...
2,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,alt.atheism,path cantaloupesrvcscmuedudasnewsharvardedunoc...,"[path, cantaloupesrvcscmuedudasnewsharvardedun...","[path, cantaloupesrvcscmuedudasnewsharvardedun...",path cantaloupesrvcscmuedudasnewsharvardedunoc...
3,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism,path cantaloupesrvcscmuedumagnesiumclubcccmued...,"[path, cantaloupesrvcscmuedumagnesiumclubcccmu...","[path, cantaloupesrvcscmuedumagnesiumclubcccmu...",path cantaloupesrvcscmuedumagnesiumclubcccmued...
4,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...,alt.atheism,xref cantaloupesrvcscmuedu altatheism talkreli...,"[xref, cantaloupesrvcscmuedu, altatheism, talk...","[xref, cantaloupesrvcscmuedu, altatheism, talk...",xref cantaloupesrvcscmuedu altatheism talkreli...



TF-IDF matrix shape: (2000, 5000)
Feature extraction complete.


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# Split the data into training and test sets
X = tfidf_matrix # Features (TF-IDF matrix)
y = df['Labels'] # Target (Categories)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nTraining set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")

# Implement and Train a Naive Bayes Classifier

# Initialize the Multinomial Naive Bayes classifier
# Multinomial Naive Bayes is suitable for text classification with TF-IDF features
naive_bayes_model = MultinomialNB()

# Train the model on the training data
print("\nTraining the Naive Bayes model...")
naive_bayes_model.fit(X_train, y_train)
print("Naive Bayes model training complete.")

# Make predictions on the test set
print("\nMaking predictions on the test set...")
y_pred = naive_bayes_model.predict(X_test)
print("Predictions complete.")

# Evaluate the model performance (Optional but recommended)
# We will evaluate the model in the next step as requested.

# The trained model (naive_bayes_model) and predictions (y_pred) are now ready for evaluation.


Training set shape: (1600, 5000)
Test set shape: (400, 5000)

Training the Naive Bayes model...
Naive Bayes model training complete.

Making predictions on the test set...
Predictions complete.


In [15]:
!pip install textblob

   ---------------------------------------- 0.0/624.3 kB ? eta -:--:--
    --------------------------------------- 10.2/624.3 kB ? eta -:--:--
   - ------------------------------------- 30.7/624.3 kB 435.7 kB/s eta 0:00:02
   ------ ------------------------------- 102.4/624.3 kB 837.8 kB/s eta 0:00:01
   ----------------- ---------------------- 266.2/624.3 kB 1.6 MB/s eta 0:00:01
   -------------------------- ------------- 409.6/624.3 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 624.3/624.3 kB 2.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ----- ---------------------------------- 0.2/1.5 MB 4.1 MB/s eta 0:00:01
   ------------ --------------------------- 0.5/1.5 MB 5.0 MB/s eta 0:00:01
   ------------------- -------------------- 0.7/1.5 MB 4.6 MB/s eta 0:00:01
   --------------------------- ------------ 1.0/1.5 MB 5.0 MB/s eta 0:00:01
   --------------------------------- ------ 1.3/1.5 MB 5.3 MB/s eta 0:00:01
   -----

In [16]:
from textblob import TextBlob

# Define a function to classify sentiment
def get_sentiment(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    if polarity > 0:
        return "Positive"
    elif polarity < 0:
        return "Negative"
    else:
        return "Neutral"

# Apply sentiment analysis to the 'Data' column
df['Sentiment'] = df['Data'].apply(get_sentiment)

# Preview the results
print(df[['Data', 'Sentiment']].head())


                                                Data Sentiment
0  Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...  Positive
1  Newsgroups: alt.atheism\nPath: cantaloupe.srv....  Negative
2  Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...  Positive
3  Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...  Positive
4  Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...  Positive


In [17]:
# Count sentiment distribution overall
sentiment_counts = df['Sentiment'].value_counts()
print("\nOverall Sentiment Distribution:")
print(sentiment_counts)

# Sentiment distribution by category (Labels)
sentiment_by_label = df.groupby('Labels')['Sentiment'].value_counts().unstack().fillna(0)
print("\nSentiment Distribution by Category:")
print(sentiment_by_label)



Overall Sentiment Distribution:
Sentiment
Positive    1543
Negative     457
Name: count, dtype: int64

Sentiment Distribution by Category:
Sentiment                 Negative  Positive
Labels                                      
alt.atheism                     23        77
comp.graphics                   24        76
comp.os.ms-windows.misc         22        78
comp.sys.ibm.pc.hardware        20        80
comp.sys.mac.hardware           24        76
comp.windows.x                  27        73
misc.forsale                    16        84
rec.autos                       17        83
rec.motorcycles                 26        74
rec.sport.baseball              29        71
rec.sport.hockey                34        66
sci.crypt                       19        81
sci.electronics                 19        81
sci.med                         29        71
sci.space                       27        73
soc.religion.christian          13        87
talk.politics.guns              30        70
talk.

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report

# Split data
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, df['Labels'], test_size=0.2, random_state=42)

# Train Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

# Predict
y_pred = nb_model.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")
print("\nDetailed Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.84
Precision: 0.85
Recall: 0.84
F1-score: 0.83

Detailed Classification Report:
                          precision    recall  f1-score   support

             alt.atheism       0.50      0.83      0.62        18
           comp.graphics       0.70      0.89      0.78        18
 comp.os.ms-windows.misc       0.91      0.91      0.91        22
comp.sys.ibm.pc.hardware       0.80      0.80      0.80        25
   comp.sys.mac.hardware       0.86      0.90      0.88        21
          comp.windows.x       0.95      0.84      0.89        25
            misc.forsale       1.00      0.78      0.88        18
               rec.autos       0.90      1.00      0.95        18
         rec.motorcycles       1.00      0.94      0.97        16
      rec.sport.baseball       0.84      0.89      0.86        18
        rec.sport.hockey       0.88      1.00      0.94        15
               sci.crypt       0.90      1.00      0.95        19
         sci.electronics       0.65      0.69    

In [ ]:
#Performance Insights:

Accuracy gives a general sense of correctness, but in multi-class problems like this, precision, recall, and F1-score are more telling.

If your F1-score is high, it means the model balances precision and recall well across categories.

In [ ]:
#Challenges Encountered:

Imbalanced classes: Some categories (like alt.atheism) may dominate, skewing predictions.

Noisy text: Blog posts often contain headers, metadata, or irrelevant tokens that dilute signal.

Short or vague posts: These may lack enough context for meaningful classification.

Overfitting: If TF-IDF features are too sparse or high-dimensional, the model might overfit.

In [ ]:
#Reflection on Sentiment Analysis Results
Sentiment Distribution:

If most blog posts are neutral, it suggests factual or discussion-based content.

A high number of positive posts may indicate enthusiastic or supportive tones.

Negative sentiments could reflect criticism, frustration, or controversial topics.

In [ ]:
#Implications:

Categories like talk. politics, the Middle East, or alt. atheism might show more polarized sentiments.

Sentiment analysis can reveal emotional undercurrents that classification alone misses—useful for understanding audience tone or topic sensitivity.